# AOC2022

## Day 12 / Part 1 / Hill Climbing Algorithm

Problem Description: https://adventofcode.com/2022/day/12

Input: [Example](aoc2022_day12_example.txt)

### Notes
- The heightmap can simply be visualized (e.g. [here](aoc2022_day12_example.txt) or, with a text editor).

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 12, part 1."""
import logging
import sys
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra
import numpy as np

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day12_example.txt"

In [3]:
def build_graph(heightmap):
    """
    Build adjacency matrix from heightmap.

    Every row of the adjacency matrix contains all indices of the heightmap:
    heightmap index:           adjacency graph:
    [  0,  1,  2,  4,          [  0, 1, 2, 3, ... 17]
       5,  6,  7,  8,     -->     ...
       9, 10, 11, 12,             ...
      13, 14, 15, 16, 17]         0, 1, 2, 3, ... 17]
    """
    graph = np.zeros(
        (
            heightmap.shape[0]*heightmap.shape[1],
            heightmap.shape[0]*heightmap.shape[1]
        )
    )
    for y_pos in range(heightmap.shape[0]):
        for x_pos in range(heightmap.shape[1]):
            src_height = heightmap[y_pos, x_pos]
            for y_delta, x_delta in [(1, 0), (0, -1), (0, 1), (-1, 0)]:
                if (
                    (
                        y_pos + y_delta < 0 or
                        heightmap.shape[0] <= y_pos + y_delta
                    ) or (
                        x_pos + x_delta < 0 or
                        heightmap.shape[1] <= x_pos + x_delta
                    ) or (
                        y_delta == x_delta == 0
                    )
                ):
                    continue
                trg_height = heightmap[y_pos+y_delta, x_pos+x_delta]
                if trg_height <= src_height+1:
                    graph[
                        y_pos * heightmap.shape[1] + x_pos,
                        (y_pos+y_delta) * heightmap.shape[1] + (x_pos+x_delta)
                    ] = 1
    return graph


def reconstr_path(predecessors, end_idx, map_width):
    """
    Reconstruct a path of coordinates from a map with index predecessors,
    ending with end_idx.
    """
    path = [(end_idx // map_width, end_idx % map_width)]
    idx = end_idx
    while idx >= 0:
        path.append((idx // map_width, idx % map_width))
        idx = predecessors[idx]
    return list(reversed(path))

In [4]:
def main():
    """Main function to solve puzzle."""
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        heightmap = np.asarray(
            [list(line.rstrip()) for line in file_obj.readlines()]
        )

        start_pos = None
        end_pos = None
        for y_pos in range(heightmap.shape[0]):
            for x_pos in range(heightmap.shape[1]):
                if heightmap[y_pos, x_pos] == "S":
                    start_pos = (y_pos, x_pos)
                    heightmap[y_pos, x_pos] = "a"
                elif heightmap[y_pos, x_pos] == "E":
                    end_pos = (y_pos, x_pos)
                    heightmap[y_pos, x_pos] = "z"

        heightmap = np.asarray(
            list(map(lambda row: [ord(cell)-97 for cell in row], heightmap))
        )
        graph = csr_matrix(build_graph(heightmap))

    start_idx = start_pos[0]*heightmap.shape[1] + start_pos[1]
    end_idx = end_pos[0]*heightmap.shape[1] + end_pos[1]
    dist_matrix, predecessors = dijkstra(
        csgraph=graph,
        directed=True,
        unweighted=False,
        indices=start_idx,
        return_predecessors=True
    )

    LOGGER.debug(
        "path %s\n",
        reconstr_path(predecessors, end_idx, heightmap.shape[1])
    )

    print(f"solution: {dist_matrix[end_idx]}")

In [5]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 31.0
